In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline


class Config:
    def __init__(self):
        pass


def set_config(config_dict):
    config = Config()
    config.__dict__ = config_dict
    return config


In [2]:
config = set_config({
    'row_start': 7228,
    'row_end': 7288,
    "timesteps": 2,
    "max_batch_size": 1,
    "layer_size": 2,
    "unit_size": 1,
    "dropout": 0,
    "learning_rate": 0.1,
    "max_epochs": 1,
    "time_col": 'tanggal',
    "feature": ['rr']
})


def train_test_split(dataset, time_step=1):
    dataX, dataY = [], []
    # for i in range(len(dataset)-time_step-1):
    for i in range(len(dataset)-time_step):
        dataX.append(dataset[i:(i+time_step), 0])
        dataY.append(dataset[i+time_step, 0])
    return np.array(dataX), np.array(dataY)


In [3]:
dataset = pd.read_csv('../Data/1985-2021.csv').replace(to_replace=[8888, 9999, 2555], value=np.nan)[config.row_start:config.row_end]
dataset.interpolate(inplace=True)

datelist = [datetime.strptime(date, '%Y-%m-%d').date() for date in list(dataset[config.time_col])]

In [4]:
featureset = dataset[config.feature]
vector_featureset = featureset.values

scaller = MinMaxScaler()
vector_featureset_scaled = scaller.fit_transform(vector_featureset)

train_size = int(vector_featureset_scaled.size * 0.9)
trainset, testset = vector_featureset_scaled[0:train_size], vector_featureset_scaled[train_size:vector_featureset_scaled.size]

X_train, y_train = train_test_split(trainset, time_step=config.timesteps)
X_test, y_test = train_test_split(testset, time_step=config.timesteps)

X_train = np.reshape(X_train, (X_train.shape[0], len(config.feature), X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], len(config.feature), X_test.shape[1]))

In [5]:
# y_pred_loss = list()
# y_true_loss = list()
# loss = list()


def mean_squared_error(y_true, y_pred):
    # y_pred_loss.append(y_pred)
    # y_true_loss.append(y_true)
    # loss.append(tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true)))
    return tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true))


def root_mean_squared_error(y_true, y_pred):
    return tf.keras.backend.sqrt(mean_squared_error(y_pred, y_true))


model = tf.keras.models.Sequential()

for i in range(0, config.layer_size):
    model.add(
        tf.keras.layers.LSTM(
            units=config.unit_size,
            return_sequences=False if i == config.layer_size - 1 else True,
            batch_input_shape=(config.max_batch_size, len(config.feature), config.timesteps),
            go_backwards=True,
            dropout=config.dropout,
            # weights=[
            #     np.array([
            #         [0.5774, 0.5774, 0.5774, 0.5774],
            #         [0.5774, 0.5774, 0.5774, 0.5774]
            #     ]),
            #     np.array([
            #         [0.5774, 0.5774, 0.5774, 0.5774]
            #     ]),
            #     np.zeros([4])
            # ]
        )
    )
else:
    model.compile(
        optimizer=tf.keras.optimizers.SGD(learning_rate=config.learning_rate),
        loss=mean_squared_error,
        run_eagerly=True
    )
    model.summary()
    
print(model.layers[1].get_weights())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (1, 1, 1)                 16        
                                                                 
 lstm_1 (LSTM)               (1, 1)                    12        
                                                                 
Total params: 28
Trainable params: 28
Non-trainable params: 0
_________________________________________________________________
[array([[ 0.96612954,  0.5867425 ,  0.5229045 , -0.3984329 ]],
      dtype=float32), array([[ 0.26341128, -0.02856461,  0.92797273, -0.26204062]],
      dtype=float32), array([0., 1., 0., 0.], dtype=float32)]


In [6]:
# from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

batch_loss = list()


history = model.fit(
    X_train,
    y_train,
    shuffle=False,
    epochs=config.max_epochs,
    verbose=1,
    batch_size=config.max_batch_size,
    callbacks=[
        tf.keras.callbacks.LambdaCallback(
            on_epoch_begin=None,
            on_epoch_end=None,
            on_batch_begin=None,
            on_batch_end=lambda batch, logs=None: batch_loss.append(logs['loss']),
            on_train_begin=None,
            on_train_end=None,
        )
    ]
)


52/52 [==============================] - 2s 40ms/step - loss: 0.0964


In [7]:
batch_loss

[1.000861406326294,
 0.6438044309616089,
 0.5689262747764587,
 0.4417469799518585,
 0.355497270822525,
 0.3002797067165375,
 0.25738605856895447,
 0.22528739273548126,
 0.20195704698562622,
 0.18210595846176147,
 0.17124976217746735,
 0.1582501381635666,
 0.15931902825832367,
 0.14880333840847015,
 0.13899938762187958,
 0.13146844506263733,
 0.1250218003988266,
 0.12078744918107986,
 0.11789202690124512,
 0.1143995150923729,
 0.10899513214826584,
 0.10464102774858475,
 0.10095265507698059,
 0.09674959629774094,
 0.10980334877967834,
 0.10558585077524185,
 0.10437968373298645,
 0.11295115947723389,
 0.11014033854007721,
 0.11484004557132721,
 0.11116813123226166,
 0.1077926829457283,
 0.10471247881650925,
 0.10172402113676071,
 0.0988866463303566,
 0.1128869354724884,
 0.1099226176738739,
 0.10869266092777252,
 0.10604213178157806,
 0.10354752838611603,
 0.10106594115495682,
 0.10332410782575607,
 0.10094572603702545,
 0.09876785427331924,
 0.096573106944561,
 0.10517173260450363,
 0.10